In [17]:
# o código do gosdt usa várias funções antigas do sklearn,
# gerando muitas notificações; vamos desligar os warnings
# para 'limpar' a saída dessas mensagens
import warnings
warnings.filterwarnings('ignore')

In [18]:
import pandas as pd
import numpy as np
import pathlib
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
# Alteração - correção dos nomes de pacotes
from gosdt.model.threshold_guess import compute_thresholds, cut
from gosdt import GOSDT

# Example 3
FICO contains more than 10,000 samples and 23 continuous features. Binarizing these continuous features leads to a data matrix with more than 1,900 features. In this example, we show how threshold and lower bound guesses help GOSDT find a near optimal tree on the FICO dataset. 

For this example, we start by using guessing immediately, which is what we recommend you do if you have an extraordinarily large number of features. We _could_ have followed the same process as in example 2 (first run without guessing, then see what happens and use guessing if we time out). However, there are so many features here that we will definitely need guessing to find accurate trees in reasonable time and memory. If we tried running without guessing, we would be likely to kill the kernel because of how much memory we would be using!

In [19]:
# read the dataset
df = pd.read_csv("../data/fico/fico.csv")
X, y = df.iloc[:,:-1], df.iloc[:,-1]
h = df.columns[:-1]
df

,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,MaxDelq2PublicRecLast12M,...,MSinceMostRecentInqexcl7days,NumInqLast6M,NumInqLast6Mexcl7days,NetFractionRevolvingBurden,NetFractionInstallBurden,NumRevolvingTradesWBalance,NumInstallTradesWBalance,NumBank2NatlTradesWHighUtilization,PercentTradesWBalance,RiskPerformance
0,55,144,4,84,20,3,0,83,2,3,...,0,0,0,33,-8,8,1,1,69,1
1,61,58,15,41,2,4,4,100,-7,0,...,0,0,0,0,-8,0,-8,-8,0,1
2,67,66,5,24,9,0,0,100,-7,7,...,0,4,4,53,66,4,2,1,86,1
3,66,169,1,73,28,1,1,93,76,6,...,0,5,4,72,83,6,4,3,91,1
4,81,333,27,132,12,0,0,100,-7,7,...,0,1,1,51,89,3,1,0,80,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10454,73,131,5,57,21,0,0,95,80,6,...,7,0,0,26,-8,5,2,0,100,0
10455,65,147,39,68,11,0,0,92,28,6,...,1,1,1,86,53,2,2,1,80,1
10456,74,129,6,64,18,1,1,100,-7,6,...,3,4,4,6,-8,5,-8,0,56,1
10457,72,234,12,113,42,2,2,96,35,6,...,6,0,0,19,-8,4,1,0,38,1


In [20]:
# If train-test split is desired
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2021)
print(X_train.shape)
print(X_test.shape)

(8367, 23)
(2092, 23)


#### 1. Threshold guess: we first use the gradient boosted tree to guess useful thresholds. 

In [21]:
# GBDT parameters for threshold and lower bound guesses
n_est = 40
max_depth = 1

In [22]:
# guess thresholds
X_train = pd.DataFrame(X_train, columns=h)
X_test = pd.DataFrame(X_test, columns=h)
X_train_guessed, thresholds, header, threshold_guess_time = compute_thresholds(X_train.copy(), y_train, n_est, max_depth)
X_test_guessed = cut(X_test.copy(), thresholds)
X_test_guessed = X_test_guessed[header]
print(X_train_guessed.shape)
print(X_test_guessed.shape)
print("train set column names == test set column names: {}".format(list(X_train_guessed.columns)==list(X_test_guessed.columns)))

(8367, 24)
(2092, 24)
train set column names == test set column names: True


#### 2. Lower bound guess: we use the gradient boosted tree to get labels for the training dataset. 

In [23]:
# guess lower bound
import time
start_time = time.perf_counter()
clf = GradientBoostingClassifier(n_estimators=n_est, max_depth=max_depth, random_state=42)
clf.fit(X_train_guessed, y_train.values.flatten())
warm_labels = clf.predict(X_train_guessed)

elapsed_time = time.perf_counter() - start_time

lb_time = elapsed_time

In [24]:
# save the labels as a tmp file and return the path to it.
labelsdir = pathlib.Path('/tmp/warm_lb_labels')
labelsdir.mkdir(exist_ok=True, parents=True)

labelpath = labelsdir / 'warm_label.tmp'
labelpath = str(labelpath)
pd.DataFrame(warm_labels).to_csv(labelpath, header="class_labels",index=None) # TODO: verify this formats correctly for gosdt (shouldn't require headers)


#### 3. train GOSDT model

In [25]:
# train GOSDT model
config = {
            "regularization": 0.001,
            "depth_budget": 5,
            "time_limit": 60,
            "warm_LB": True,
            "path_to_labels": labelpath,
            "similar_support": False
        }

model = GOSDT(config)

model.fit(X_train_guessed, pd.DataFrame(y_train))

print("evaluate the model, extracting tree and scores", flush=True)

gosdt reported successful execution
training completed. 0.000/0.000/1.343 (user, system, wall), mem=0 MB
bounds: [0.289495..0.289495] (0.000000) loss=0.283495, iterations=20601
evaluate the model, extracting tree and scores


#### 4. Get results

In [26]:
# get the results
train_acc = model.score(X_train_guessed, y_train)
test_acc = model.score(X_test_guessed, y_test)
n_leaves = model.leaves()
n_nodes = model.nodes()
time = model.utime

print("Model training time: {}".format(time))
print("Training accuracy: {}".format(train_acc))
print("Test accuracy: {}".format(test_acc))
print("# of leaves: {}".format(n_leaves))
print(model.tree)

Model training time: 0.0
Training accuracy: 0.7165053185132066
Test accuracy: 0.7084130019120459
# of leaves: 6
if ExternalRiskEstimate<=67.5 = 1 then:
    predicted class: 1
    misclassification penalty: 0.101
    complexity penalty: 0.001

else if ExternalRiskEstimate<=67.5 != 1 and MSinceMostRecentInqexcl7days<=-7.5 = 1 then:
    predicted class: 0
    misclassification penalty: 0.005
    complexity penalty: 0.001

else if ExternalRiskEstimate<=67.5 != 1 and ExternalRiskEstimate<=74.5 = 1 and MSinceMostRecentInqexcl7days<=-7.5 != 1 and MSinceMostRecentInqexcl7days<=0.5 = 1 then:
    predicted class: 1
    misclassification penalty: 0.053
    complexity penalty: 0.001

else if ExternalRiskEstimate<=67.5 != 1 and ExternalRiskEstimate<=74.5 = 1 and MSinceMostRecentInqexcl7days<=-7.5 != 1 and MSinceMostRecentInqexcl7days<=0.5 != 1 then:
    predicted class: 0
    misclassification penalty: 0.026
    complexity penalty: 0.001

else if ExternalRiskEstimate<=67.5 != 1 and ExternalRiskEsti

As you can see, we find a reasonably accurate model quite quickly on datasets as complicated as FICO!

Thank you for reading our tutorial. Please do try out our methods with different parameters and datasets. Happy tree training!